<a href="https://colab.research.google.com/github/Olfeng-xalaz/SCM/blob/main/Raffinerieproblem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
# pip als Paketmanager
! pip install -q pyscipopt

In [44]:
from pyscipopt import Model, quicksum

# Optimierungsmodell zur Produktionsplanung

In [45]:
#Erstellen einer Modellinstanz
scip = Model()

## Indexmengen

In [46]:
J = ['K1','K2','K3','K4',] # Menge der Kraftstoffe
I = ['R1','R2','R3','RafCap'] # Menge der Rohöle und maximale Raffineriekapazität

# Entscheidungsvariablen

In [47]:
x = {} # Wie viel stelle ich von meinen Kraftstoffen her?
for j in J: # Die Schleife läuft über jedes Produkt
    x[j] = scip.addVar(vtype='C', lb=0, ub=None, name=j) # vtype --> Für jedes Proudkt wird eine Variable der Reellen Zahlne C angelegt, lb = 0 --> Lower Bound, ub = Kein Upper Bound, name = j --> Variable bekommt den Namen des Produkts
print('Entscheidungsvariablen =', scip.getVars()) # Gibt alle im Modell gespeicherten Variablen aus

Entscheidungsvariablen = [K1, K2, K3, K4]


# Parameter

In [48]:
c = {} # DB der Kraftstfoffe
c ['K1'] = 5
c ['K2'] = 8
c ['K3'] = 15
c ['K4'] = 3

cap = {} # max. Kapazität
cap ['R1'] = 1000
cap ['R2'] = 500
cap ['R3'] = 1500
cap ['RafCap'] = 2250

a = {} # Menge an Rohöl pro Kraftstoff
a['R1']={'K1':1,'K2':1,'K3':2,'K4':1}
a['R2']={'K1':2,'K2':1,'K3':3,'K4':0}
a['R3']={'K1':0,'K2':1,'K3':2,'K4':1}
a['RafCap']={'K1':1,'K2':1,'K3':1,'K4':1}

# Zielfunktion

In [49]:
# Maximierung des Gesamtdeckungsbeitrags
scip.setObjective(quicksum(c[j]*x[j] for j in J), sense="maximize")
# scip.setObjective --> Setzt die Zielfunktion des Optimierungsmodells
# quicksum --> bildet die Summe aller Terme

# Restriktionen

In [50]:
for i in I: # Schleife über alle Rohöle / Max. Kapazitäten
    scip.addCons(quicksum(a[i][j]*x[j] for j in J)<=cap[i], name=i) # scip.addCons(..., name=i) --> fügt die Nebenbedingung (Constraint) dem Modell hinzu
print('Nebenbedingungen =', scip.getConss()) # gibt alle erstellten Constraints aus (Kontrollausgabe)

Nebenbedingungen = [R1, R2, R3, RafCap]


# Berechnung des Ergebnisses

In [51]:
scip.optimize()
# SCIP versucht jetzt, dein Modell (Zielfunktion + Nebenbedingungen) zu lösen
status = scip.getStatus() # Status, Sagt dir, was der Solver erreicht hat,
gap = scip.getGap() # Der Optimalitäts-Gap ist wichtig vor allem bei MILP: Gap = 0 → optimal bewiesen, Gap > 0 → du hast eine Lösung, aber evtl. noch nicht garantiert optimal
print(f"Status des Solvers: {status} , GAP={gap}% \n") # printen

if status == "optimal":
    print('LÖSUNG:')
    print('Zielfunktionswert (Deckungsbeitrag) =', scip.getObjVal()) # Gibt den optimalen Zielfunktionswert zurück, also den maximalen Deckungsbeitrag:
    for j in J: # Gibt den optimalen Wert (getVal) der Variablen x[j] zurück, also wie viel von welchem Kraftstoff produziert werden soll
        print(f'{j} =', round(scip.getVal(x[j]))) # Rundet die Ausgaben.
else:
    print('Problem hat keine Lösung') # Wenn sich keine optimale Lösung ergibt

# Anmerkung: 👉 getStatus() gibt den Lösungsstatus des Solvers zurück, z. B.:
# optimal
# infeasible
# unbounded
# timelimit
# unknown
# Diese Statuswerte sind in SCIP intern definiert. Deswegen muss die Bedeutung von optimal nicht erst deklariert werden.


Status des Solvers: optimal , GAP=0.0% 

LÖSUNG:
Zielfunktionswert (Deckungsbeitrag) = 5500.0
K1 = 0
K2 = 500
K3 = 0
K4 = 500
